In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import h2o
import psutil
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

## Logistic Regression (with H2O Frame)

In [2]:
pct_memory=4
virtual_memory=psutil.virtual_memory()
min_mem_size=int(round(int(pct_memory*virtual_memory.available)/1073741824,0))
print(min_mem_size)

34


In [3]:
h2o.init(strict_version_check=False,max_mem_size=min_mem_size)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "10.0.2" 2018-07-17; Java(TM) SE Runtime Environment 18.3 (build 10.0.2+13); Java HotSpot(TM) 64-Bit Server VM 18.3 (build 10.0.2+13, mixed mode)
  Starting server from /Users/mandy/Library/Python/3.7/lib/python/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/5l/qqvypzcs2sjbbnhb777lxczh0000gn/T/tmpcher9rsp
  JVM stdout: /var/folders/5l/qqvypzcs2sjbbnhb777lxczh0000gn/T/tmpcher9rsp/h2o_mandy_started_from_python.out
  JVM stderr: /var/folders/5l/qqvypzcs2sjbbnhb777lxczh0000gn/T/tmpcher9rsp/h2o_mandy_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.1
H2O_cluster_version_age:,16 days
H2O_cluster_name:,H2O_from_python_mandy_2bq0d8
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,34 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"accepting new members, healthy"


In [4]:
# Load data into H2O
train = h2o.import_file('Data/train.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
train.head()

C1,MONTH,DAY,DAY_OF_WEEK,DEPARTURE_TIME,AIRLINE,DISTANCE,delayed,Origin,Dest
0,1,1,4,2354,AS,1448,0,ANC,SEA
1,1,1,4,2,AA,2330,0,LAX,PBI
2,1,1,4,18,US,2296,0,SFO,CLT
3,1,1,4,15,AA,2342,0,LAX,MIA
4,1,1,4,24,AS,1448,0,SEA,ANC
5,1,1,4,20,DL,1589,0,SFO,MSP
6,1,1,4,19,NK,1299,0,LAS,MSP
7,1,1,4,44,US,2125,0,LAX,CLT
8,1,1,4,19,AA,1464,0,SFO,DFW
9,1,1,4,33,DL,1747,0,LAS,ATL


In [6]:
split = train["C1"].quantile(prob = [0.8])[1]
train["Train"] = (train["C1"] < split).ifelse("Yes", "No")
Train = train[train["Train"] == "Yes"]
Test = train[train["Train"] == "No"]

In [7]:
predictors = ['MONTH', 'DAY', 'DAY_OF_WEEK', 'DEPARTURE_TIME', 'AIRLINE', 'DISTANCE', 'Origin', 'Dest', 'Train']
response = 'delayed'

In [8]:
import time
st = time.time()
glm_logistic_plus_summary = H2OGeneralizedLinearEstimator(family = "binomial")
glm_logistic_plus_summary.train(x=predictors,
                               y= response, 
                               training_frame = Train, 
                               validation_frame = Test, 
                               model_id="glm_logistic")
t = time.time() - st
print('Runs %0.3f seconds.' % t)

glm Model Build progress: |███████████████████████████████████████████████| 100%
Runs 6.485 seconds.


In [9]:
print("AUC: " + str(round(glm_logistic_plus_summary.auc(valid = True), 3)))

AUC: 0.658


In [10]:
h2o.cluster().shutdown()

H2O session _sid_9f03 was not closed properly.


## Random Forest (one or more cores)

In [11]:
train =pd.read_csv('Data/train.csv')

In [12]:
train.drop(columns = ['Unnamed: 0'], inplace=True)

In [13]:
train = pd.get_dummies(train)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(train.drop(['delayed'],axis=1), train.delayed, 
                                                    test_size=0.33, random_state=42, stratify=train.delayed)

In [15]:
import time
st = time.time()
clf = RandomForestClassifier(n_estimators=100, max_depth=10, n_jobs=1, verbose = 2)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_pred, y_test))
t = time.time() - st
print('Runs %0.3f seconds.' % t)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.1s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 12.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   14.7s finished


0.8229635510385692
Runs 770.467 seconds.


In [16]:
roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   14.6s finished


0.7025768034992487

In [17]:
s = time.time()
clf_m = RandomForestClassifier(n_estimators=100, max_depth=10, n_jobs=4, verbose = 2)
clf_m.fit(X_train, y_train)
y_pred_m = clf_m.predict(X_test)
print(accuracy_score(y_pred_m, y_test))
tt = time.time() - s
print('Runs %0.3f seconds.' % tt)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 100building tree 2 of 100building tree 3 of 100building tree 4 of 100



building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100

[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  1.5min



building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 7

[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  4.3min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    4.5s finished


0.8229635510385692
Runs 275.183 seconds.


In [18]:
roc_auc_score(y_test, clf_m.predict_proba(X_test)[:, 1])

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    4.5s finished


0.7016396856883113

In [19]:
s = time.time()
clf_m = RandomForestClassifier(n_estimators=100, max_depth=10, n_jobs=8, verbose = 2)
clf_m.fit(X_train, y_train)
y_pred_m = clf_m.predict(X_test)
print(accuracy_score(y_pred_m, y_test))
tt = time.time() - s
print('Runs %0.3f seconds.' % tt)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 100building tree 2 of 100building tree 3 of 100building tree 4 of 100



building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100

[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  1.5min



building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 7

[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  4.2min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    4.4s finished


0.8229635510385692
Runs 268.721 seconds.


In [20]:
roc_auc_score(y_test, clf_m.predict_proba(X_test)[:, 1])

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    4.4s finished


0.7040093978783989

In [21]:
s = time.time()
clf_m = RandomForestClassifier(n_estimators=100, max_depth=10, n_jobs=12, verbose = 2)
clf_m.fit(X_train, y_train)
y_pred_m = clf_m.predict(X_test)
print(accuracy_score(y_pred_m, y_test))
tt = time.time() - s
print('Runs %0.3f seconds.' % tt)

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.


building tree 1 of 100building tree 2 of 100building tree 3 of 100building tree 4 of 100building tree 5 of 100building tree 6 of 100building tree 7 of 100

building tree 8 of 100building tree 9 of 100building tree 10 of 100building tree 11 of 100building tree 12 of 100









building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100


[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   46.7s


building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70

[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:  3.4min finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.7s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    3.1s finished


0.8229635510385692
Runs 214.648 seconds.


In [22]:
roc_auc_score(y_test, clf_m.predict_proba(X_test)[:, 1])

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.7s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    3.1s finished


0.7015803516487025